In [43]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

from project_huishoudboekje.config import GeneralSettings
from project_huishoudboekje.app import App

In [4]:
df = pd.read_excel(GeneralSettings.project_path / 'data/processed/transactions.xlsx').sort_values(
            by='DATE', ascending=False)

df_analysis = App().prepare_data(df)
df_budget = App().prepare_data_budget(df_analysis['DATE'].max())

c:\users\robsc\documents\python\huishoudboekje\src\project_huishoudboekje\app.py:678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: -x['AMOUNT'] if ((x['FINANCIAL_TYPE'] == 'credit') & (x['GROUP'] != 'Inkomsten')) | (
c:\users\robsc\documents\python\huishoudboekje\src\project_huishoudboekje\app.py:680: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\robsc\documents\python\huishoudboekje\src\project_huishoudboekje\app.py:684: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [10]:
df_analysis_stack = df_analysis.groupby(['YEAR_MONTH', 'INCOME_IND'])['AMOUNT_NW'].sum().reset_index().rename(
    columns={'AMOUNT_NW': 'ACTUAL'})
df_budget_stack = df_budget.groupby(['YEAR_MONTH', 'INCOME_IND'])['AMOUNT_NW'].sum().reset_index().rename(
    columns={'AMOUNT_NW': 'BUDGET'})

In [53]:
df_total_stack = df_budget_stack.merge(df_analysis_stack, on=['YEAR_MONTH', 'INCOME_IND'], how='left').fillna(0)

In [58]:
df_total_stack['DELTA'] = df_total_stack.apply(lambda x: x['BUDGET'] - x['ACTUAL'] if x['INCOME_IND'] == 'Uitgaven' else x['ACTUAL'] - x['BUDGET'], axis=1) #df_total_stack['BUDGET'] - df_total_stack['ACTUAL']

In [46]:
df_total_stack

,YEAR_MONTH,INCOME_IND,BUDGET,ACTUAL,DELTA,COLORMAP
0,2022-01,Inkomsten,2501,4050.26,1549.26,#1f77b4
1,2022-01,Uitgaven,28342,9312.28,-37654.28,#ff7f0e
2,2022-02,Inkomsten,3250,9022.26,5772.26,#1f77b4
3,2022-02,Uitgaven,32626,4733.51,-37359.51,#ff7f0e
4,2022-03,Inkomsten,3844,4047.76,203.76,#1f77b4
5,2022-03,Uitgaven,34297,8616.51,-42913.51,#ff7f0e
6,2022-04,Inkomsten,3036,4048.69,1012.69,#1f77b4
7,2022-04,Uitgaven,32980,15336.44,-48316.44,#ff7f0e
8,2022-05,Inkomsten,3700,3991.69,291.69,#1f77b4
9,2022-05,Uitgaven,30902,3298.64,-34200.64,#ff7f0e


In [55]:
df_stack_pivot = pd.pivot_table(
    df_total_stack, values='DELTA', index=['YEAR_MONTH'], columns=['INCOME_IND'], aggfunc=np.sum).fillna(0).reset_index()

In [56]:
df_stack_pivot

INCOME_IND,YEAR_MONTH,Inkomsten,Uitgaven
0,2022-01,1549.26,-37654.28
1,2022-02,5772.26,-37359.51
2,2022-03,203.76,-42913.51
3,2022-04,1012.69,-48316.44
4,2022-05,291.69,-34200.64
5,2022-06,1710.94,-34033.77
6,2022-07,3175.51,-39431.02
7,2022-08,-2432.00,-33784.81


In [60]:
fig = px.bar(df_total_stack, x='YEAR_MONTH', y='DELTA', color='INCOME_IND', barmode='group',
                      color_discrete_sequence=["#B6E880", "#EF553B"])
fig.add_hline(y=0)
fig.update_layout(title='Delta Budget-Actual', showlegend=False, yaxis_title=None,
                  xaxis_title=None, margin=dict(l=20, r=20, t=30, b=20))
#fig.update_traces(marker={'color': df_total_stack['COLORMAP']}, selector=dict(type='bar'))

In [40]:
df = pd.DataFrame({'idx':['A','B','C'], 'cat1':[0.98,0.92,0.91], 'cat2':[0.93,0.80,0.87], 'cat3':[0.55,0.865,0.955], 'cat4':[1,1,0.96]})


fig = go.Figure()
indexes = ['A','B','C']
groups = ['cat1','cat2','cat3','cat4']
colors = ['#dc1a22','#dc1a22','#000000','#dc1a22']

## this can be generalized a bit better
width = 0.2
offset = [-0.4,-0.2,0,0.2]

for i in indexes:
    for g, c, o in zip(groups, colors, offset):
            fig.add_trace(
                go.Bar(x=df['idx'], y=df[g], marker_color=c, width=width, offset=o)
            )

fig.update_traces(
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5, 
    opacity=0.6
)

fig.update_layout(
    xaxis_title='idx',
    yaxis_title='value',
    barmode='group', 
    showlegend=False
)

fig.show()


In [ ]:
fig = go.Figure()
indexes = df_stack_pivot['YEAR_MONTH']
groups = ['Inkomsten', 'Uitgaven']
colors = ['#dc1a22','#000000']

## this can be generalized a bit better
width = 0.2
offset = [-0.4,-0.2,0,0.2]

for i in indexes:
    for g, c, o in zip(groups, colors, offset):
            fig.add_trace(
                go.Bar(x=df['idx'], y=df[g], marker_color=c, width=width, offset=o)
            )

fig.update_traces(
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5, 
    opacity=0.6
)

fig.update_layout(
    xaxis_title='idx',
    yaxis_title='value',
    barmode='group', 
    showlegend=False
)

fig.show()